# Finetuning of Neural Network Model for a New Site

In [54]:
import pandas as pd
import numpy as np
from joblib import load, dump
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance

import sys
sys.path.append('../')
import config

data_path = Path.home()/'OneDrive'/'Kirstin'/'Uni'/'Year4'/'MSciProject'/'data_files'/'saved_files'

In [55]:
site = 'MHD'
site_name = config.site_dict[site]

transferred_site = 'GSN'
transferred_site_name = config.site_dict[transferred_site]

compound = config.compound

print(f"Finetuning a neural network model based on \033[1m{site_name}\033[0;0m to be applicable at \033[1m{transferred_site_name}\033[0;0m.")

Finetuning a neural network model based on Mace Head, Ireland to be applicable at Gosan, South Korea.


### Loading in Model & Initialising Data

In [56]:
# loading in model
original_model = load(data_path/f'nn_model_{site}.joblib')

# loading in training data
original_data = pd.read_csv(data_path/f'for_model_{compound}_{site}.csv', parse_dates=['time'])

training = original_data[(original_data['time'].dt.year >= 2016) & (original_data['time'].dt.year <= 2018)]
training = training.drop(columns=['time'])

X_train = training.drop(columns=['flag'])
y_train = training['flag']
y_train = y_train.dropna()
X_train = X_train.loc[y_train.index]

# training model on original site training data
original_model.fit(X_train, y_train)

MLPClassifier(alpha=0.05, batch_size=100, beta_2=0.9, early_stopping=True,
              max_iter=1000, random_state=42, shuffle=False)

In [57]:
# loading in data
data = pd.read_csv(data_path/f'for_model_{compound}_{transferred_site}.csv', parse_dates=['time'])
data.sample(5)

,time,flag,u10_0,u10_1,u10_2,u10_3,u10_4,u10_5,u10_6,u10_7,...,v500_2_past,v500_3_past,v500_4_past,v500_5_past,v500_6_past,v500_7_past,v500_8_past,v500_13_past,v500_14_past,v500_15_past
3174,2010-07-15 23:00:00,1.0,-0.690763,-2.413504,-2.079362,-0.467843,-4.827441,0.580885,1.065391,2.663067,...,2.956724,8.791500,3.919943,2.058213,2.199645,2.840251,10.080106,1.347354,4.139025,-2.066434
7768,2014-09-21 03:00:00,1.0,-5.385603,1.209845,-3.363423,-1.770733,-3.291179,-2.176362,-2.439491,-5.905895,...,-4.130726,-3.561815,6.215248,13.754383,11.021891,-4.304200,-3.337700,13.703741,-10.480322,-5.818061
608,2008-03-07 07:00:00,1.0,1.867782,2.919305,0.042532,2.486858,4.190870,-2.035694,-4.202700,-2.256213,...,-1.197463,-5.641761,-0.776241,-0.254065,-1.059376,0.290158,-14.883116,0.587219,6.950803,-0.691532
8985,2016-01-20 05:00:00,1.0,1.994573,2.263357,-0.303354,3.497597,5.162735,-1.191066,-3.149698,-3.520934,...,-9.148509,-23.608479,-7.080099,3.852128,-6.611067,-3.859116,-7.364631,-8.681700,-8.706151,-2.002994
8535,2015-06-08 09:00:00,0.0,-2.749644,2.936976,0.297533,-0.440825,1.947626,4.742805,2.025746,-1.811198,...,-14.069165,-4.372243,-0.317615,-2.377529,-1.225105,-1.639202,-2.731714,-1.069158,-0.342285,-0.717616


In [58]:
# setting up data for finetuning
train_data_ft = data[(data['time'].dt.year >= 2013) & (data['time'].dt.year <= 2014)]
val_data_ft = data[(data['time'].dt.year >= 2015) & (data['time'].dt.year <= 2015)]
test_data_ft = data[(data['time'].dt.year >= 2016) & (data['time'].dt.year <= 2017)]

print(f"Train range: {train_data_ft['time'].min()} -> {train_data_ft['time'].max()}. Length: {len(train_data_ft)}")
print(f"Val range: {val_data_ft['time'].min()} -> {val_data_ft['time'].max()}. Length: {len(val_data_ft)}")
print(f"Test range: {test_data_ft['time'].min()} -> {test_data_ft['time'].max()}. Length: {len(test_data_ft)}")

train_data_ft = train_data_ft.drop(columns=['time'])
val_data_ft = val_data_ft.drop(columns=['time'])
test_data_ft = test_data_ft.drop(columns=['time'])

X_train_ft = train_data_ft.drop(columns=['flag'])
y_train_ft = train_data_ft['flag']
X_val_ft = val_data_ft.drop(columns=['flag'])
y_val_ft = val_data_ft['flag']
X_test_ft = test_data_ft.drop(columns=['flag'])
y_test_ft = test_data_ft['flag']

y_train_ft = y_train_ft.dropna()
y_val_ft = y_val_ft.dropna()
y_test_ft = y_test_ft.dropna()

X_train_ft = X_train_ft.loc[y_train_ft.index]
X_val_ft = X_val_ft.loc[y_val_ft.index]
X_test_ft = X_test_ft.loc[y_test_ft.index]

Train range: 2013-01-01 01:00:00 -> 2014-12-31 23:00:00. Length: 2004
Val range: 2015-01-01 01:00:00 -> 2015-12-31 19:00:00. Length: 812
Test range: 2016-01-01 21:00:00 -> 2017-12-31 23:00:00. Length: 2012


In [59]:
# setting up data for full retuning
train_data = data[(data['time'].dt.year >= 2011) & (data['time'].dt.year <= 2013)]
val_data = data[(data['time'].dt.year >= 2014) & (data['time'].dt.year <= 2014)]
test_data = data[(data['time'].dt.year >= 2015) & (data['time'].dt.year <= 2017)]

print(f"Train range: {train_data['time'].min()} -> {train_data['time'].max()}. Length: {len(train_data)}")
print(f"Val range: {val_data['time'].min()} -> {val_data['time'].max()}. Length: {len(val_data)}")
print(f"Test range: {test_data['time'].min()} -> {test_data['time'].max()}. Length: {len(test_data)}")

train_data = train_data.drop(columns=['time'])
val_data = val_data.drop(columns=['time'])
test_data = test_data.drop(columns=['time'])

X_train = train_data.drop(columns=['flag'])
y_train = train_data['flag']
X_val = val_data.drop(columns=['flag'])
y_val = val_data['flag']
X_test = test_data.drop(columns=['flag'])
y_test = test_data['flag']

y_train = y_train.dropna()
y_val = y_val.dropna()
y_test = y_test.dropna()

X_train = X_train.loc[y_train.index]
X_val = X_val.loc[y_val.index]
X_test = X_test.loc[y_test.index]

Train range: 2011-01-01 11:00:00 -> 2013-12-30 09:00:00. Length: 3186
Val range: 2014-01-03 19:00:00 -> 2014-12-31 23:00:00. Length: 933
Test range: 2015-01-01 01:00:00 -> 2017-12-31 23:00:00. Length: 2824


### Testing Existing Model on Data

In [60]:
y_val_pred = original_model.predict(X_val)

precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f'{site} Model Precision: {precision:.2f}')
print(f'{site} Model Recall: {recall:.2f}')
print(f'{site} Model F1: {f1:.2f}')

MHD Model Precision: 0.87
MHD Model Recall: 0.80
MHD Model F1: 0.83


### Finetuning Model

In [61]:
# allowing warm start & therefore fine-tuning
original_model.warm_start = True

# adding more iterations to the model
original_model.max_iter += 250

# fitting the model to the new data (one years worth)
original_model.fit(X_train_ft, y_train_ft)

MLPClassifier(alpha=0.05, batch_size=100, beta_2=0.9, early_stopping=True,
              max_iter=1250, random_state=42, shuffle=False, warm_start=True)

In [62]:
# evaluating model on validation set
y_val_pred = original_model.predict(X_val_ft)

precision = precision_score(y_val_ft, y_val_pred)
recall = recall_score(y_val_ft, y_val_pred)
f1 = f1_score(y_val_ft, y_val_pred)

print(f'Finetuned Model Precision: {precision:.2f}')
print(f'Finetuned Model Recall: {recall:.2f}')
print(f'Finetuned Model F1: {f1:.2f}')

Finetuned Model Precision: 0.94
Finetuned Model Recall: 0.21
Finetuned Model F1: 0.35


In [63]:
# exploring the distribution of the predictions
y_val_pred_int = y_val_pred.astype(int)

counts = np.bincount(y_val_pred_int)

print("Number of non-baselines:", counts[0])
print("Number of baselines:", counts[1])

assert counts[0] > 0, "Model has predicted no non-baselines. This is likely due to the model being overfit."

Number of non-baselines: 681
Number of baselines: 131


In [64]:
# saving model
dump(original_model, data_path/f'nn_model_{transferred_site}_finetuned.joblib')

['C:\\Users\\kirst\\OneDrive\\Kirstin\\Uni\\Year4\\MSciProject\\data_files\\saved_files\\nn_model_GSN_finetuned.joblib']

### Retuning Model Completely

In [50]:
new_model = MLPClassifier(random_state=42,
                         max_iter=1000, 
                         hidden_layer_sizes=(100,), 
                         shuffle=False,
                         activation='relu', 
                         solver='adam', 
                         alpha=0.05, 
                         learning_rate='constant', 
                         batch_size=100, 
                         early_stopping=True,
                         learning_rate_init=0.001,
                         beta_2=0.9,)

new_model.fit(X_train, y_train)

y_val_pred = new_model.predict(X_val)
y_train_pred = new_model.predict(X_train)

precision_val = precision_score(y_val, y_val_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_val = recall_score(y_val, y_val_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_val = f1_score(y_val, y_val_pred)
f1_train = f1_score(y_train, y_train_pred)

print(f'Full Retuned Model Precision (val): {precision_val:.2f}')
print(f'Full Retuned Model Precision (train): {precision_train:.2f}')
print(f'Full Retuned Model Recall (val): {recall_val:.2f}')
print(f'Full Retuned Model Recall (train): {recall_train:.2f}')
print(f'Full Retuned Model F1 (val): {f1_val:.2f}')
print(f'Full Retuned Model F1 (train): {f1_train:.2f}')

Full Retuned Model Precision (val): 0.85
Full Retuned Model Precision (train): 0.88
Full Retuned Model Recall (val): 0.93
Full Retuned Model Recall (train): 0.95
Full Retuned Model F1 (val): 0.89
Full Retuned Model F1 (train): 0.92


In [51]:
# evaluating model on test set
y_test_pred = new_model.predict(X_test)

precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print(f'Full Retuned Model Precision: {precision:.2f}')
print(f'Full Retuned Model Recall: {recall:.2f}')
print(f'Full Retuned Model F1: {f1:.2f}')

Full Retuned Model Precision: 0.86
Full Retuned Model Recall: 0.94
Full Retuned Model F1: 0.90


In [52]:
# saving model
dump(new_model, data_path/f'nn_model_{transferred_site}.joblib')

['C:\\Users\\kirst\\OneDrive\\Kirstin\\Uni\\Year4\\MSciProject\\data_files\\saved_files\\nn_model_GSN.joblib']